In [75]:
import pandas as pd


df=pd.read_csv("output/merged_output.csv", header=None)
df.columns = ['worker_id', 'question','response', 'answer', 'correct', 'confidence', 'likely', 'type']
df.correct = pd.to_numeric(df.correct, errors='coerce')
df.confidence = pd.to_numeric(df.confidence, errors='coerce')
df.likely = pd.to_numeric(df.likely, errors='coerce')
print(df.head())


        worker_id  question  response  answer  correct  confidence  likely  \
0  A2C7TMRC2QHTIW         1         1       1        1           5       4   
1  A2C7TMRC2QHTIW         2         1       0        0           6       5   
2  A2C7TMRC2QHTIW         3         0       1        0           6       4   
3  A2C7TMRC2QHTIW         4         1       0        0           7       5   
4  A2C7TMRC2QHTIW         5         0       1        0           5       4   

  type  
0  bar  
1  bar  
2  bar  
3  bar  
4  bar  


In [80]:
import scipy.stats

df['overall_accuracy'] = df['correct'].mean()
df['accuracy_question'] = df['correct'].groupby(df['question']).transform('mean')
std = df.groupby('question')['correct'].mean().std()
df['std'] = std

df['cdf'] = scipy.stats.norm(df['overall_accuracy'],df['std']).cdf(df['accuracy_question'])
df['1-cdf'] = 1 - df['cdf']
df['maxcdf'] = df[["cdf", "1-cdf"]].max(axis=1)
df['pdf'] = scipy.stats.norm(df['overall_accuracy'],df['std']).pdf(df['accuracy_question'])
print(df.head())

        worker_id  question  response  answer  correct  confidence  likely  \
0  A2C7TMRC2QHTIW         1         1       1        1           5       4   
1  A2C7TMRC2QHTIW         2         1       0        0           6       5   
2  A2C7TMRC2QHTIW         3         0       1        0           6       4   
3  A2C7TMRC2QHTIW         4         1       0        0           7       5   
4  A2C7TMRC2QHTIW         5         0       1        0           5       4   

  type  overall_accuracy  accuracy_question       std       cdf     1-cdf  \
0  bar          0.451389               0.36  0.056374  0.052496  0.947504   
1  bar          0.451389               0.52  0.056374  0.888212  0.111788   
2  bar          0.451389               0.40  0.056374  0.180997  0.819003   
3  bar          0.451389               0.52  0.056374  0.888212  0.111788   
4  bar          0.451389               0.42  0.056374  0.288833  0.711167   

     maxcdf       pdf  
0  0.947504  1.901788  
1  0.888212  3.37426

In [82]:
import statsmodels.formula.api as smf
model1 = smf.ols('confidence ~ maxcdf', df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             confidence   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.958
Date:                Wed, 12 Dec 2018   Prob (F-statistic):             0.0147
Time:                        18:07:06   Log-Likelihood:                -6822.6
No. Observations:                3600   AIC:                         1.365e+04
Df Residuals:                    3598   BIC:                         1.366e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.8270      0.148     32.581      0.0

In [83]:
model2 = smf.ols('likely ~ pdf', df).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 likely   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.521
Date:                Wed, 12 Dec 2018   Prob (F-statistic):             0.0107
Time:                        18:08:42   Log-Likelihood:                -6761.2
No. Observations:                3600   AIC:                         1.353e+04
Df Residuals:                    3598   BIC:                         1.354e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.5052      0.074     60.540      0.0

In [89]:

model3=smf.ols('correct ~ type', df).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                correct   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     19.79
Date:                Wed, 12 Dec 2018   Prob (F-statistic):           1.01e-12
Time:                        22:17:03   Log-Likelihood:                -2566.3
No. Observations:                3600   AIC:                             5141.
Df Residuals:                    3596   BIC:                             5165.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.3775      0.018  

In [90]:
df['modified_type'] = 'non-binary'
df.loc[df['type'] == 'box','modified_type'] = 'binary'
df.loc[df['type'] == 'bar','modified_type'] = 'binary'

model4=smf.ols('correct ~ modified_type', df).fit()
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:                correct   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.842
Date:                Wed, 12 Dec 2018   Prob (F-statistic):             0.0501
Time:                        22:17:27   Log-Likelihood:                -2593.8
No. Observations:                3600   AIC:                             5192.
Df Residuals:                    3598   BIC:                             5204.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         